In [1]:
#проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе.

In [2]:
import pandas as pd

import seaborn as sns
import matplotlib.pylab as plt
import os

%matplotlib inline

In [3]:
path_to_folder = 'subsid'

In [4]:
list_name_file = os.listdir(path_to_folder)

In [5]:
list_name_file

['prod_activations_logs.csv',
 'tm_sales_1.csv',
 'tm_sales_2.csv',
 'tm_sales_3.csv']

In [6]:
df = pd.DataFrame(columns=['FILIAL_ID', 'SUBS_ID', 'PROD_ID', 'ACT_DTTM'])

In [7]:
for item in list_name_file:
    if item != 'prod_activations_logs.csv':
        df_new = pd.read_csv("".join([path_to_folder, item]), sep=';')
        df = df.append(df_new)
    

In [8]:
df = df.reset_index()

In [9]:
df

,index,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM
0,0,1,id4651830,1954,20-03-2020 14:59
1,1,5,7646509,6431,19-03-2020 13:00
2,2,4,id7412683,3313,22-03-2020 17:25
3,0,3,id5416547,1743,17-03-2020 10:17
4,1,4,NaN,7421,12-03-2020 11:25
5,2,7,8362218,9879,05-03-2020 11:42
6,3,2,2185490,3210,16-03-2020 16:28
7,0,3,id5764122,1499,18-03-2020 15:44
8,1,6,id7642700,3020,15-03-2020 14:21
9,2,2,id1374509,5677,17-03-2020 11:48


In [10]:
df = df.astype({'FILIAL_ID':'int64', 'PROD_ID':'int64'})
df['ACT_DTTM'] = pd.to_datetime(df.ACT_DTTM, format='%d-%m-%Y %H:%M')

In [11]:
df

,index,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM
0,0,1,id4651830,1954,2020-03-20 14:59:00
1,1,5,7646509,6431,2020-03-19 13:00:00
2,2,4,id7412683,3313,2020-03-22 17:25:00
3,0,3,id5416547,1743,2020-03-17 10:17:00
4,1,4,NaN,7421,2020-03-12 11:25:00
5,2,7,8362218,9879,2020-03-05 11:42:00
6,3,2,2185490,3210,2020-03-16 16:28:00
7,0,3,id5764122,1499,2020-03-18 15:44:00
8,1,6,id7642700,3020,2020-03-15 14:21:00
9,2,2,id1374509,5677,2020-03-17 11:48:00


In [12]:
df = df.dropna(subset=['SUBS_ID'])

In [13]:
df

,index,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM
0,0,1,id4651830,1954,2020-03-20 14:59:00
1,1,5,7646509,6431,2020-03-19 13:00:00
2,2,4,id7412683,3313,2020-03-22 17:25:00
3,0,3,id5416547,1743,2020-03-17 10:17:00
5,2,7,8362218,9879,2020-03-05 11:42:00
6,3,2,2185490,3210,2020-03-16 16:28:00
7,0,3,id5764122,1499,2020-03-18 15:44:00
8,1,6,id7642700,3020,2020-03-15 14:21:00
9,2,2,id1374509,5677,2020-03-17 11:48:00


In [14]:
def add_id(item):
    if not item.startswith('id'):
        return 'id'+item
    return item

In [15]:
df.SUBS_ID = df.SUBS_ID.apply(add_id)

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [16]:
df_2 = pd.read_csv("".join([path_to_folder, 'prod_activations_logs.csv']), sep=';')

In [36]:
df_2.dtypes


SUBS_ID       object
PROD_ID        int64
START_DTTM    object
END_DTTM      object
dtype: object

In [18]:
df = df.merge(df_2, on=['SUBS_ID', 'PROD_ID'], how='inner')

In [19]:
df['START_DTTM'] = pd.to_datetime(df.START_DTTM, format='%d-%m-%Y %H:%M')
df['END_DTTM'] = pd.to_datetime(df.END_DTTM, format='%d-%m-%Y %H:%M')

In [20]:
df

,index,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM
0,0,1,id4651830,1954,2020-03-20 14:59:00,2020-03-20 14:59:00,2020-12-01 00:00:00
1,1,5,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:00:00,2020-03-19 13:03:00
2,0,3,id5416547,1743,2020-03-17 10:17:00,2020-03-17 10:17:00,2020-03-25 11:00:00
3,3,2,id2185490,3210,2020-03-16 16:28:00,2020-03-16 16:28:00,2020-12-01 00:00:00
4,1,6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 14:21:00,2020-03-15 23:42:00


In [21]:
#продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)

In [22]:
df['timedelta_column'] = df.END_DTTM - df.START_DTTM


In [23]:
df

,index,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,timedelta_column
0,0,1,id4651830,1954,2020-03-20 14:59:00,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,1,5,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,0,3,id5416547,1743,2020-03-17 10:17:00,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,3,2,id2185490,3210,2020-03-16 16:28:00,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
4,1,6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [38]:
df_new = df[df.timedelta_column >= '5m']

In [45]:
df_new = df_new.sort_values(by='SUBS_ID', ascending=True)

In [46]:
", ".join(df_new.SUBS_ID.to_list())

'id2185490, id4651830, id5416547, id7642700'

In [37]:
df_new


,index,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,timedelta_column
1,1,5,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00


In [27]:
df_.to_csv('out.csv', index=False)  

NameError: name 'df_' is not defined